In [34]:
from moco.builder import MoCo
import torch
from torchvision import models

moco=MoCo(
    models.__dict__['resnet18'],
    128, 65536, 0.999,
    contr_tau=0.2,
    align_alpha=None,
    unif_t=None,
    unif_intra_batch=True,
    mlp=True)
moco=moco.cuda()
checkpoint = torch.load('../moco/save/custom_imagenet_n02106550/mocom0.999_contr1tau0.2_mlp_aug+_cos_b256_lr0.06_e120,160,200/checkpoint_0199.pth.tar', map_location=torch.device('cuda:0'))

# moco=moco.load_state_dict(checkpoint['state_dict'])
# moco=checkpoint

In [35]:
# moco=moco.load_state_dict(checkpoint['state_dict'],strict=False)

In [36]:
# moco

In [37]:
state_dict =checkpoint['state_dict']
from collections import OrderedDict
new_state_dict = OrderedDict()

for k, v in state_dict.items():
    if 'module' in k:
        k = k.split('.')[1:]
        k = '.'.join(k)
    new_state_dict[k]=v

moco.load_state_dict(new_state_dict)

<All keys matched successfully>

In [39]:
moco.encoder_q

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  